In [ ]:
library(pheatmap)
library(RColorBrewer)
library(ComplexHeatmap)
library(circlize)
library(ggpubr)
library(viridis)
library(dendextend)
library(data.table)

### Figure 3b

In [ ]:
tpm <- read.csv("CNV_sum_filter.csv", header = TRUE)
row.names(tpm) <- make.unique(tpm[, 1])
tpm <- tpm[, -1]
tpm[is.na(tpm)] <- 0

cluster <- read.csv("cnv_clu5_th2.csv", header=T)
rownames(cluster) <- cluster[, 1]
cluster <- subset(cluster, select = -X)
head(cluster)


In [ ]:
mapping_dict <- c('1' = 'CIN-low', '2' = 'Chr8q-amp', '3' = 'Chr20-amp', '4' = 'Chr18-amp', '5' = 'CIN-moderate')
cluster$Cluster <- mapping_dict[as.character(cluster$Cluster)]

sorted_data <- cluster[order(factor(cluster$Cluster, levels = c('Chr8q-amp', 'Chr20-amp', 'Chr18-amp', 'CIN-moderate', 'CIN-low'))), , drop = FALSE]
sorted_expr_matrix <- tpm[, row.names(sorted_data)]

cluster_count <- read.csv("cluster_count.csv", header=T)
sorted_index <- rownames(sorted_data)
merged_data <- merge(cluster_count, sorted_data, by.x = "X", by.y = "row.names")
merged_data <- merged_data[order(match(merged_data$X, sorted_index)), ]


In [ ]:
mat <- as.matrix(sorted_expr_matrix)

cell_count <- merged_data$Cell.Count
column_widths <- ifelse(cell_count / 1000 < 5, 0, round(cell_count / 1000))
expanded_mat <- do.call(cbind, lapply(1:ncol(mat), function(i) {
    if (column_widths[i] != 0) {
        matrix(rep(mat[, i], column_widths[i]), ncol = column_widths[i])
    } else {
        NULL
    }
}))
mat <- expanded_mat

In [ ]:
cancer_type <- read.csv("HD-OV 100.csv", header=1)[,c(2,4)]
col_names <- colnames(as.matrix(sorted_expr_matrix))
tumor_types <- vector("character", length(col_names))

for (i in seq_along(col_names)) {
    sample_id <- strsplit(col_names[i], "_")[[1]][1]
    match_index <- match(sample_id, cancer_type[, 1])
    if (!is.na(match_index)) {
        tumor_types[i] <- cancer_type[match_index, 2]
    } else {
        tumor_types[i] <- NA
    }
}

adjusted_tumor_types <- unlist(lapply(1:length(tumor_types), function(i) {
    if (column_widths[i] != 0) {
        rep(tumor_types[i], column_widths[i])
    } else {
        NULL
    }
}))

tumor_type_colors <- c("OC" = "#ab526a", "EC" = "#d68081", "CC" = "#2878b5")
annotation <- HeatmapAnnotation(
  tumor_type = adjusted_tumor_types,
  col = list(tumor_type = tumor_type_colors),
  annotation_legend_param = list(
    tumor_type = list(
      at = c("OC", "EC", "CC"),
      labels = c("OC", "EC", "CC")
    )
  )
)

In [ ]:
my_palette <- colorRamp2(c(-0.005, 0, 0.02), c("#483d8b", "#ffffff", "#800000"))
row_groups <- rep(1:22, c(1889, 1145, 999, 703, 813, 926, 825, 609, 712, 681, 1200, 948, 299, 580, 524, 753, 1067, 257, 1301, 507, 191, 403))

value_counts <- table(sorted_data$Cluster)
sum1 <- sum(column_widths[1:14])
sum2 <- sum(column_widths[15:29])
sum3 <- sum(column_widths[30:65])
sum4 <- sum(column_widths[66:275])
sum5 <- sum(column_widths[276:809])
col_groups <- rep(c('high cnv-1', 'high cnv-2', 'high cnv-3', 'moderate cnv', 'low cnv'), c(sum1,sum2,sum3,sum4,sum5))
col_groups <- factor(col_groups, levels = c('high cnv-1', 'high cnv-2', 'high cnv-3', 'moderate cnv', 'low cnv'))

ht_list <- Heatmap(
        mat,
        name = "CNV Score",
        cluster_rows = FALSE,
        cluster_columns = FALSE,
        col = my_palette,
        column_split = col_groups,
        row_split = row_groups,
        gap = unit(0.5, "mm"),
        show_row_names = FALSE,
        show_column_names = FALSE,
        top_annotation = annotation
)

print(ht_list)

### Figure 3c

In [ ]:
chr_len <- read.csv("./ref/hg38.list", sep='\t',header=FALSE)

chr_length <- chr_len$V2
names(chr_length) = chr_len$V1
chr_offset = c(0, cumsum(as.numeric(chr_length)))

In [ ]:
HD_sample_info <- read.csv("HD-OV 100.csv")
name_list <- HD_sample_info$sample_id

In [ ]:
path <- "./cn/raw"
name_list <- HD_sample_info$sampleName
file_list <- paste0(path, "/", name_list, "_gainLoss.bed")
segs <- list()
seg_length_list <- list()
i <- 0

for (file_name in file_list) {
    i <- i + 1
    
    if (file.exists(file_name)) {
        sample_name <- sub("_.*", "", basename(file_name))
        seg_data <- read.table(file_name, header = TRUE, sep = "\t", stringsAsFactors = FALSE)

        seg_data$chrom <- gsub("chr", "", seg_data$chrom)
        seg_data <- seg_data[(seg_data$end - seg_data$start) > 100000, ]
        seg_length <- sum(seg_data$end - seg_data$start)
        seg_data$total_cn <- ifelse(seg_data$total_cn > 9, 9, seg_data$total_cn)

        segs[[i]] <- seg_data
        seg_length_list[[i]] <- list(length = seg_length, index = i)
    } else {
        cat("File does not exist:", file_name, "\n")
    }
}

df_total_length <- do.call(rbind, lapply(seg_length_list, as.data.frame))
df_total_length_sorted <- df_total_length[order(-df_total_length$length), ]

In [ ]:
segs_order = segs[df_total_length_sorted$index]

In [ ]:
plot(0, 0,
	type="n",
	xlim=c(0, sum(chr_length)),
	ylim=c(0, length(segs)),
	bty="n",
	xaxt="n", xlab="Chromosome",
	yaxt="n", ylab="Tumour"
)

poly_height = 1

cp_colors = c(
	rev(brewer.pal(5, "Blues")[c(4, 5)]), # 0, 1
	rgb(150, 150, 150, maxColorValue=255),
	brewer.pal(8, "Reds")[c(-1, -2)]
)

for (i in 1:length(segs_order)) {
    if (nrow(segs_order[[i]]) > 0) {
        chr = segs_order[[i]]$chrom
        start = segs_order[[i]][["start"]]
        end = segs_order[[i]][["end"]]
        cp_num = segs_order[[i]][["total_cn"]]

        for (j in 1:nrow(segs_order[[i]])) {
            offset = chr_offset[as.integer(chr[j])]
            genome_start = offset + start[j]
            genome_end = offset + end[j]

            polygon(
                c(genome_start, genome_end, genome_end, genome_start),
                c(i, i, i + poly_height, i + poly_height),
                border=NA,
                col=cp_colors[cp_num[j] + 1]
            )
        }
    }
}

for (chr_end in chr_offset) {
	abline(v=chr_end,
		col="black"
	)
}
par(xpd=TRUE)

for (chr in 1:(length(chr_offset) - 1)) {
	mid_point = mean(chr_offset[c(chr, chr + 1)])
	text(mid_point, 1, labels=chr, pos=1)
}

rect(
  xleft = 0,
  ybottom = 1,
  xright = sum(chr_length),
  ytop = 100 + poly_height,
  border = "black",
  lwd = 2
)
